# Step 0

Necessary imports + *Scala* interpreter

In [ ]:
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.apache.spark._
import java.io.{ PrintWriter, File, FileOutputStream }

sc

Intitializing Scala interpreter ...

Spark Web UI available at http://captain01.os.hpc.tuwien.ac.at:9999/proxy/application_1683456198714_2383
SparkContext available as 'sc' (version = 3.2.3, master = yarn, app id = application_1683456198714_2383)
SparkSession available as 'spark'


import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.apache.spark._
import java.io.{PrintWriter, File, FileOutputStream}
res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@6ee8b1ce


In [ ]:
// Importing the files required for the processing.

In [4]:
val stopwords = sc.textFile("stopwods.txt").collect()
val rdd = sc.textFile("hdfs:///user/dic23_shared/amazon-reviews/full/reviews_devset.json")

stopwords: Array[String] = Array(a, aa, able, about, above, absorbs, accord, according, accordingly, across, actually, after, afterwards, again, against, ain, album, album, all, allow, allows, almost, alone, along, already, also, although, always, am, among, amongst, an, and, another, any, anybody, anyhow, anyone, anything, anyway, anyways, anywhere, apart, app, appear, appreciate, appropriate, are, aren, around, as, aside, ask, asking, associated, at, available, away, awfully, b, baby, bb, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, believe, below, beside, besides, best, better, between, beyond, bibs, bike, book, books, both, brief, bulbs, but, by, c, came, camera, can, cannot, cant, car, case, cause, causes, cd, certain, certainly, changes,...


The resulting data RDD will contain tuples where the first element is the compacted "category" value and the second element is the compacted "reviewText" value extracted from the JSON rows in the original RDD.

In [5]:
val data = rdd.map{row => val json_row = parse(row)
(compact(json_row \ "category"),
 compact(json_row \ "reviewText"))}

data: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[8] at map at <console>:34


# Cleaning

The underscore ('_') is a placeholder used in Scala for an anonymous function or partial function. It is often used when you want to refer to the input parameters of a function without explicitly naming them.

In [6]:
val pairs = data.flatMap{ x =>
    val category = x._1;
    val reviews = x._2.toLowerCase.replaceAll("[()\\[\\]{}.!?,;:+='-_\\\"`~#@&*%€$§/\1234567890\t]"," ")
        .split(" ")
        .map(_.trim)
        .distinct
        .filter(!stopwords.contains(_)).filter(_.length > 1)
    reviews.map{ term => (term,category)}}

pairs: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[9] at flatMap at <console>:35


The following line could be used to print the data thats cleaned and grouped together based on our requirement.
Commenting the line, as this was just a part of validation.

In [7]:
// println(pairs.collect().mkString("\n"))

Counting will be relevant for *ChiSquare*. First  we count the pairs, then the terms, then the reviews per categories (for simplicity I called it *categories*) and finally we count the amount of reviews. `collect()` retrieves all the elements of the *RDD*. Note that besides *reviews* all other *vals* are *RDD* "objects", where as *reviews* is simply a *double*.

In [8]:
val pairsCount = pairs.map(x => (x,1))
    .reduceByKey((x,y) => x+y)
val terms = pairs.collect{case ((term,category)) => (term,1)}
    .reduceByKey((x,y) => x+y)
val categories = data.collect{case ((category,review)) => (category,1)}
    .reduceByKey((x,y) => x+y)
val reviews = categories.values.sum()

pairsCount: org.apache.spark.rdd.RDD[((String, String), Int)] = ShuffledRDD[11] at reduceByKey at <console>:36
terms: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[14] at reduceByKey at <console>:38
categories: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[17] at reduceByKey at <console>:40
reviews: Double = 78829.0


In [9]:
// println(terms.collect().mkString("\n"))

# Chi-square Test

$x^2 = \frac{n(ad - bc)^2}{(a+b)(a+c)(b+d)(c+d)}$

**a** ... number of occurences of pair category and term

**b** ... occurences of term - number of occurences of pair category and term

**c** ... number of reviews in a category - number of occurences of pair category and term

**d** ... number of reviews - number of reviews in a category - occurences of term - number of occurences of pair category and term

In [10]:
val A = (pairsCount.map {case ((term, category), count_p) => ((category,term), count_p)})

val B = terms.join(A.map 
    {case ((category, term),count_p) => (term, (category, count_p))}).map {
    case (term,(count_t ,(category,count_p))) => ((category,term),(count_t -count_p))}

val C = categories.join(A.map 
    {case ((category, term),count_p) => (category, (term, count_p))}).map {
    case (category,(count_c ,(term,count_p))) => ((category,term),(count_c -count_p))}

val D = categories.join(B.map{
    case ( (category,term),b) => (category,(term,b))}).map{
    case (category,(count_c,(term,b))) => ((category,term),(reviews-count_c-b))}

A: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[20] at map at <console>:37
B: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[25] at map at <console>:40
C: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[30] at map at <console>:44
D: org.apache.spark.rdd.RDD[((String, String), Double)] = MapPartitionsRDD[35] at map at <console>:48


## Actual calculation

We reproduce the above mentioned equation.

In [11]:
val calculation = (A.join(B).join(C).join(D))
    .map {case ((term, category),(((a,b),c),d)) => ((term,category),(reviews*(scala.math.pow((a*d - b*c),2)/((a+b) *(a+c) *(b+d)*(c+d)))))}

calculation: org.apache.spark.rdd.RDD[((String, String), Double)] = MapPartitionsRDD[45] at map at <console>:39


In [ ]:
Taking the top 75 as it was done in exe1 so that the comparision can be done appropriately.

In [12]:
val grouped = calculation.map{
    case ((category,term),t) => (category,(term,t))}
    .groupByKey()
    .sortBy(_._1)
    .mapValues(value => value.toArray.sortWith(_._2 > _._2).take(75))

grouped: org.apache.spark.rdd.RDD[(String, Array[(String, Double)])] = MapPartitionsRDD[53] at mapValues at <console>:38


# Finalization

In [13]:
val finalTest = grouped.map
    {case(category, term_test) => (category.replace("\"", "") + " " ) + term_test.mkString(" ")
    .replaceAll("[\\(\\)]", "")
    .replace(",", ": ")}.collect()

finalTest: Array[String] = Array(Apps_for_Android games: 3081.149337484292 play: 2158.3694068201294 graphics: 1505.51089773515 kindle: 1470.820942569012 addictive: 1311.905562727777 challenging: 1038.128455852793 coins: 1002.6647889526382 addicting: 990.8441134974868 fire: 956.1470053110605 levels: 825.3813282736015 playing: 692.9340396014182 ads: 642.3969794099202 puzzles: 596.7716753070063 apps: 548.7810653104153 free: 500.9884786241356 bingo: 409.2358492981346 mahjong: 322.00891943980963 download: 303.8649278202287 faotd: 288.8577201586641 facebook: 282.5170543702901 downloaded: 262.77022492215735 hints: 242.61029019440056 solitaire: 211.64299578381858 android: 211.5810584959861 puzzle: 198.855822173525 gameplay: 198.5123356770461 freezes: 189.67737127837006 unlock: 185.7521008338788...


In [14]:
val merged = grouped.flatMap{
    case (key,value) => value}.map(_._1).collect().distinct.sorted

merged: Array[String] = Array(acdelco, acne, acoustic, acre, acted, acting, action, actor, actors, actress, acura, adapter, addario, addicted, addicting, addictive, adjustment, adorable, ads, adventure, aftertaste, aired, airsoft, akai, albums, almonds, alpha, alternator, altima, ammo, amp, amplitube, android, animated, animation, anime, answering, antenna, ants, appetite, apple, apply, apps, aquarium, ar, arch, arnley, aroma, arrangements, articulation, artisan, artist, artists, asus, atv, audio, author, authors, avent, avery, awesome, babies, back, backpacking, bag, bait, baking, ball, ballad, ballads, ballasts, balls, band, bands, banjo, barbie, barking, barrel, bass, bassinet, bath, bathroom, batteries, battery, bbq, beam, beard, bearings, bed, behringer, believable, berserker, bib,...


## Output generation

In [15]:
val txtfile = new PrintWriter(new File( "output_rdd.txt" ))
finalTest.foreach( value => {txtfile.write(value + "\n")})
merged.foreach( value => {txtfile.write(value + " " )} )
txtfile.close()

txtfile: java.io.PrintWriter = java.io.PrintWriter@1b66e84c
